In [7]:
import os

import rasterio
import geopandas as gpd
import numpy as np


/Users/d3y010/.pyenv/versions/py3.9.4_cerf/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [44]:
root_dir = "/Users/d3y010/projects/cerf/suitability/data"

template_raster = os.path.join(root_dir, "reference", "cerf_landmask_0-land.tif")
conus_boundary_file = os.path.join(root_dir, "reference", "cerf_conus_boundary.shp")

raw_dir = os.path.join(root_dir, "raw")

in_shp = os.path.join(raw_dir, "BIA_National_LAR_shp", "BIA_National_LAR.shp")

out_shp = os.path.join(raw_dir, "bia_national_lar_conus.shp")

# create output raster name
output_raster_temp = os.path.join(raw_dir, "cerf_bia_tribal_lands_temp.tif")

output_raster = os.path.join(root_dir, "technology-specific", "cerf_bia_tribal_lands_2019.tif")


In [45]:
with rasterio.open(template_raster) as src:
    
    metadata = src.meta.copy()
    
    target_crs = src.crs
    
    arr_template = src.read()


In [32]:
gdf = gpd.read_file(in_shp).to_crs(target_crs)


In [33]:
# add suitability field to rasterize
gdf['suitable'] = 1


In [34]:
conus_gdf = gpd.read_file(conus_boundary_file).to_crs(target_crs)


In [35]:
gdf_conus = gpd.clip(gdf, conus_gdf)


In [36]:
gdf_conus.head()

,LARID,LARName,GISAcres,Classifica,Shape_STAr,Shape_STLe,geometry,suitable
0,LAR0001,Cheyenne River LAR,5.697394e+06,1,1.313910,8.262795,"MULTIPOLYGON (((-351679.503 899671.175, -35152...",1
1,LAR0002,Crow Creek LAR,5.751046e+05,1,0.134807,2.597780,"POLYGON ((-272263.394 762950.226, -272278.577 ...",1
2,LAR0003,Flandreau LAR,4.564290e+03,1,0.001071,0.355752,"MULTIPOLYGON (((-45624.475 737464.668, -46031....",1
3,LAR0004,Fort Berthold LAR,2.241221e+06,1,0.492177,3.306533,"POLYGON ((-509880.357 1189467.082, -509707.251...",1
4,LAR0005,Lake Traverse (Sisseton) LAR,2.323396e+05,1,0.053071,23.092728,"MULTIPOLYGON (((-100386.721 924612.471, -10038...",1


In [37]:
gdf_conus[["suitable", "geometry"]].to_file(out_shp)



In [47]:

# get basename from files
base_noext = os.path.splitext(os.path.basename(out_shp))[0]

gdal_rasterize_cmd = f"gdal_rasterize -l {base_noext} -a suitable -tr 1000.0 1000.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {out_shp} {output_raster_temp}"

print(gdal_rasterize_cmd, '\n')

os.system(gdal_rasterize_cmd)


gdal_rasterize -l bia_national_lar_conus -a suitable -tr 1000.0 1000.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/projects/cerf/suitability/data/raw/bia_national_lar_conus.shp /Users/d3y010/projects/cerf/suitability/data/raw/cerf_bia_tribal_lands_temp.tif 



0

In [49]:
with rasterio.open(output_raster, 'w', **metadata) as dest:
    
    with rasterio.open(output_raster_temp) as src:
        
        arr = src.read()
        
        arr += arr_template
        
        dest.write(arr)
        
        
        
        